In [2]:
import pandas as pd

try:
    df = pd.read_csv('datasets/SIOE 2024-01-31.csv', encoding='latin-1', delimiter=';')
    print(df.info())
except pd.errors.ParserError as e:
    print("Error parsing CSV file:", e)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8797 entries, 0 to 8796
Data columns (total 49 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   classe                                   8797 non-null   object
 1   tipo                                     8797 non-null   object
 2   código SIOE                              8797 non-null   int64 
 3   sigla                                    8797 non-null   object
 4   designação                               8797 non-null   object
 5   path                                     8797 non-null   object
 6   cae                                      8797 non-null   object
 7   sigla do ministério/secretaria regional  8797 non-null   object
 8   nome do ministério/secretaria regional   8797 non-null   object
 9   âmbito                                   8797 non-null   object
 10  endereço                                 8797 non-null   obj

In [9]:
#count the number of rows
print("Number of rows:", len(df))

#size grouped by distrito

print(df.groupby('distrito').size())
print("\n")
print(df.groupby('classe').size())
print("\n")
print(df.groupby('tipo').size())
print("\n")
print(df.groupby('cae').size())



Number of rows: 8797
distrito
 -                        3
Aveiro                  418
Beja                    230
Braga                   620
Bragança                364
Castelo Branco          275
Coimbra                 412
Faro                    300
Guarda                  384
Ilha Terceira           130
Ilha da Graciosa         23
Ilha da Madeira         343
Ilha das Flores          34
Ilha de Porto Santo      14
Ilha de Santa Maria      26
Ilha de São Jorge        37
Ilha de São Miguel      274
Ilha do Corvo            14
Ilha do Faial            68
Ilha do Pico             46
Leiria                  322
Lisboa                 1340
Portalegre              194
Porto                   803
Santarém                385
Setúbal                 327
Viana do Castelo        341
Vila Real               350
Viseu                   504
Évora                   216
dtype: int64


classe
Entidade        6199
Sub-Entidade    2598
dtype: int64


tipo
CN         750
CN+JUR    8047
dtype: int64


c

In [10]:
#save the data to a new file csv file iso-8859-1
df.to_csv('datasets/SIOE 2024-01-31-utf8.csv', encoding='utf-8', index=False)

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8797 entries, 0 to 8796
Data columns (total 49 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   classe                                   8797 non-null   object
 1   tipo                                     8797 non-null   object
 2   código SIOE                              8797 non-null   int64 
 3   sigla                                    8797 non-null   object
 4   designação                               8797 non-null   object
 5   path                                     8797 non-null   object
 6   cae                                      8797 non-null   object
 7   sigla do ministério/secretaria regional  8797 non-null   object
 8   nome do ministério/secretaria regional   8797 non-null   object
 9   âmbito                                   8797 non-null   object
 10  endereço                                 8797 non-null   obj

In [4]:
import pandas as pd

df = pd.read_csv('datasets/SIOE 2024-01-31-utf8.csv', encoding='utf-8')

#nova coluna com o código postal em que extrai os primeiros 8 digitos do código postal original
df['codigo_postal'] = df['código postal'].str.extract(r'(\d{4}-\d{3})')

print(df.info())

#save the data to a new file csv file iso-8859-1
df.to_csv('datasets/SIOE 2024-01-31-utf8.csv', encoding='utf-8', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8797 entries, 0 to 8796
Data columns (total 50 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   classe                                   8797 non-null   object
 1   tipo                                     8797 non-null   object
 2   código SIOE                              8797 non-null   int64 
 3   sigla                                    8797 non-null   object
 4   designação                               8797 non-null   object
 5   path                                     8797 non-null   object
 6   cae                                      8797 non-null   object
 7   sigla do ministério/secretaria regional  8797 non-null   object
 8   nome do ministério/secretaria regional   8797 non-null   object
 9   âmbito                                   8797 non-null   object
 10  endereço                                 8797 non-null   obj

In [11]:
import pandas as pd
import requests

# Suponho que já tens um DataFrame df com uma coluna 'codigo_postal'

# Atualiza a função fetch_lat_lon para a versão corrigida
def fetch_lat_lon(codigo_postal, api_key):
    if '-' in codigo_postal:
        try:
            cp4, cp3 = codigo_postal.split('-')
            url = f'https://www.cttcodigopostal.pt/api/v1/{api_key}/{cp4}-{cp3}'
            
            response = requests.get(url)
            if response.status_code == 200:  # Sucesso na resposta
                data = response.json()
                if data:  # Verifica se a lista de dados não está vazia
                    first_item = data[0]
                    if 'latitude' in first_item and 'longitude' in first_item:
                        return first_item['latitude'], first_item['longitude']
                    else:
                        return None, None
                else:
                    return None, None
            else:
                return None, None
        except Exception as e:
            print(f"Erro ao processar o código postal {codigo_postal}: {e}")
            return None, None
    else:
        return None, None

# Certifica-te de que a tua API-KEY está corretamente definida
api_key = 'bc10a7fe2f6744ab9719b0244d843d3b'

# Assegura que todos os códigos postais são tratados como strings
df['codigo_postal'] = df['codigo_postal'].astype(str)

# Aplica a função ao DataFrame e cria novas colunas para latitude e longitude
df[['latitude', 'longitude']] = pd.DataFrame(df['codigo_postal'].apply(lambda x: fetch_lat_lon(x, api_key)).tolist(), index=df.index)




In [10]:
def fetch_lat_lon(codigo_postal, api_key):
    if '-' in codigo_postal:
        try:
            cp4, cp3 = codigo_postal.split('-')
            url = f'https://www.cttcodigopostal.pt/api/v1/{api_key}/{cp4}-{cp3}'
            
            response = requests.get(url)
            if response.status_code == 200:  # Sucesso na resposta
                data = response.json()
                if data:  # Verifica se a lista de dados não está vazia
                    # Acessa o primeiro elemento da lista
                    first_item = data[0]
                    if 'latitude' in first_item and 'longitude' in first_item:
                        return first_item['latitude'], first_item['longitude']
                    else:
                        return None, None
                else:
                    return None, None
            else:
                return None, None
        except Exception as e:
            print(f"Erro ao processar o código postal {codigo_postal}: {e}")
            return None, None
    else:
        return None, None


# Agora, fazemos um teste com um código postal de exemplo
api_key = 'bc10a7fe2f6744ab9719b0244d843d3b' # Substitui pela tua API-KEY real
codigo_postal_teste = '1900-363' # Código postal de exemplo

# Fazemos a chamada à função com o código postal de teste
resultado_teste = fetch_lat_lon(codigo_postal_teste, api_key)
print(f"Resultado do teste: {resultado_teste}")


Resultado do teste: ('38.7254786', '-9.1170402')


In [ ]:
#save the data to a new file csv file iso-8859-1
df.to_csv('datasets/SIOE 2024-01-31-utf8.csv', encoding='utf-8', index=False)